In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

In [7]:
from bs4 import BeautifulSoup
import requests
wikiweb_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=requests.get(wikiweb_link).text

In [9]:
soup=BeautifulSoup(source,'lxml')

In [11]:
citytable=soup.find('table',class_="wikitable sortable")
dfcity = pd.read_html(str(citytable), header=0)[0]

In [22]:
dfcity['Postcode'] = dfcity['Postcode'].astype(str)
dfcity['Borough'] = dfcity['Borough'].astype(str)
dfcity['Neighbourhood'] = dfcity['Neighbourhood'].astype(str)

In [24]:
dfcity.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [28]:
dfcity=dfcity.drop(dfcity[dfcity.Borough=='Not assigned'].index)
dfcity.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [32]:
dfcity=dfcity.groupby(['Postcode',"Borough"])["Neighbourhood"].apply(lambda x: ','.join(x)).reset_index()
dfcity.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [35]:
dfcity['Neighbourhood'].loc[dfcity['Neighbourhood'].str.contains('Not assigned')]=dfcity['Borough']

wiwi=dfcity[dfcity['Postcode'].str.contains('M7A')]
wiwi

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [36]:
dfcity.shape

(103, 3)

In [72]:
from geopy.geocoders import Nominatim

In [73]:
column_names=['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Postcode,Borough,Neighborhood,Latitude,Longitude


In [74]:
for data in dfcity.itertuples():
    address = ' {0}, {1}, Ontario'.format(data[1],data[2])
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    #print( address)
    try:
        latitude = location.latitude
        longitude = location.longitude
    except: 
        latitude =0
        longitude = 0

        
    neighborhoods = neighborhoods.append({'Postcode': data[1],
                                        'Borough': data[2],
                                        'Neighborhood': data[3],
                                        'Latitude': latitude,
                                        'Longitude': longitude}, ignore_index=True)
    
neighborhoods.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.773077,-79.257774
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.773077,-79.257774
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",0.000000,0.000000
3,M1G,Scarborough,Woburn,0.000000,0.000000
4,M1H,Scarborough,Cedarbrae,0.000000,0.000000


In [75]:
neighborhoods['Latitude'].value_counts()

0.000000     85
42.858062     4
43.773077     2
43.628603     2
43.756199     1
43.662173     1
43.732822     1
43.649248     1
43.644903     1
43.718905     1
43.654374     1
43.659980     1
43.815850     1
43.763531     1
Name: Latitude, dtype: int64

In [76]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2019-04-19 04:58:25--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2019-04-19 04:58:25--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-19 04:58:29--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following

In [78]:
coorsourcedp = pd.read_csv("Geospatial_Coordinates.csv")
coorsourcedp.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:
dummydf=coorsourcedp[coorsourcedp['Postal Code']=='M1C']   
tlat=dummydf['Latitude'].values[0]
tlong=dummydf['Longitude'].values[0]
print(tlat)

43.7845351


In [70]:
column_names=['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

# instantiate new dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Make sure the data types are correct
neighborhoods['Latitude'] = neighborhoods['Latitude'].astype(np.float64)
neighborhoods['Longitude'] = neighborhoods['Longitude'].astype(np.float64)

tlat=0.0
tlong=0.0

for data in dfcity.itertuples():
    dummydf=coorsourcedp[coorsourcedp['Postal Code']==data[1]]   
    tlat=dummydf['Latitude'].values[0]
    tlong=dummydf['Longitude'].values[0]
    #print(tlat)
    neighborhoods = neighborhoods.append({'Postcode': data[1],
                                        'Borough': data[2],
                                        'Neighborhood': data[3],
                                        'Latitude': tlat,
                                        'Longitude': tlong}, ignore_index=True)

print(neighborhoods.dtypes)

Postcode         object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object


In [71]:
neighborhoods.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
